# Plotting script for e22.F1850.f09_f09_mg17 Control and CRI 263 K CESM 39 year Ensemble nudging runs
### Set up
#### Packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from scipy import stats
import matplotlib as mpl
from matplotlib import font_manager
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.mathtext import _mathtext as mathtext
import matplotlib.ticker as mticker
from matplotlib import gridspec
import matplotlib.path as mpath
import matplotlib.colors as colors
import matplotlib.dates as mdates
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import timedelta
from cmcrameri import cm
import jinja2
from Plotting_functions import wvl2wvn, wvn2wvl, p2z, z2p, t_test_two_means, Wilks_pcrit, CustomCmap, draw_circle, CalcStatSig

In [2]:
font_path = '/glade/work/glydia/conda-envs/cenv/fonts/Helvetica.ttc'  # Your font path goes here
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Helvetica'

#### Filepaths, name variables

In [ ]:
## Test numbers
tst_nums = np.arange(1,4)
tst_type = 'nudge_ensemble'

## Test names
control = 'f.e22.F1850.f09_f09_mg17.control_test_nudge_long.'
cri263K = 'f.e22.F1850.f09_f09_mg17.cri263K_test_nudge_long.'

## Significance type
sig = 'Wilks'

## Time averaging type
time_avg = 3    # 0: Monthly, 1: Yearly, 2: Seasonal, 3: All data

## Ensemble mean or All members
ens_type = 'Mean'

## Filtering
filter = False
filter_str = 'filtered' if filter else 'non_filtered'

## Filepaths
path_to_data = '/glade/work/glydia/Arctic_CRI_processed_data/processed_wind_nudging_ensemble_data/'
path_to_graphs = '/glade/u/home/glydia/wind_nudging_40yr_ensemble_graphs/'+filter_str+'/'

## Variables to process
var_list = np.array(['FLDS','T','TS','U','V','Target_U','Target_V','OPTS_MAT'])
var = var_list[0]

## Plot types to make
map_type = True

In [4]:
## Chunking variables
la_chunk = 64
lo_chunk = 96
le_chunk = 4

In [5]:
%%time

## Select plot type - yearly or monthly - to make and assign variables accordingly
# Seasonal
if time_avg == 2:
    time_str = 'Season'

# All-data average
elif time_avg == 3:
    time_str = 'All_data'
    
elif time_avg == 0:
    time_str = 'Month'
    
elif time_avg == 4:
    time_str = 'Timeseries'
    
elif time_avg == 1:
    time_str = 'Year'

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


#### Load data

In [6]:
%%time

## Load data
# Load control ensemble members
control_data_avg = xr.open_dataset(path_to_data+control+var+'.avg.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', 
                                   chunks={'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})
if time_avg < 4:
    control_data_std = xr.open_dataset(path_to_data+control+var+'.std.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', 
                                       chunks={'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})
    n_control = xr.open_dataset(path_to_data+control+var+'.n.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', 
                                chunks={'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})

CPU times: user 1.19 s, sys: 191 ms, total: 1.38 s
Wall time: 12.8 s


In [7]:
%%time

# Load cri263K ensemble members
cri263K_data_avg = xr.open_dataset(path_to_data+cri263K+var+'.avg.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', 
                                   chunks={'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})

if time_avg < 4:
    cri263K_data_std = xr.open_dataset(path_to_data+cri263K+var+'.std.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', 
                                       chunks={'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})
    n_cri263K = xr.open_dataset(path_to_data+cri263K+var+'.n.'+ens_type+'.'+time_str+'.'+filter_str+'.nc', 
                                chunks={'lat':la_chunk,'lon':lo_chunk,'lev':le_chunk})

CPU times: user 8.1 ms, sys: 4.05 ms, total: 12.1 ms
Wall time: 68.4 ms


#### Define custom colobars

In [8]:
# Difference colorbars
tppr_levels = np.linspace(-3,3,21)
tppr_levels[10] = -0.000000001
flux_levels = np.linspace(-10,10,21)
lwp_levels = np.linspace(-75,75,21)
iwp_levels = flux_levels
lcc_levels = np.linspace(-.5,.5,21)
wcf_levels = flux_levels
wnd_levels = flux_levels

vik_cmap = cm.vik

# Create cmap and norms
diff_cmap, tppr_norm = CustomCmap(tppr_levels,vik_cmap,[],False)
_, flux_norm = CustomCmap(flux_levels,vik_cmap,[],False)
_, lwp_norm = CustomCmap(lwp_levels,vik_cmap,[],False)
_, iwp_norm = CustomCmap(iwp_levels,vik_cmap,[],False)
_, lcc_norm = CustomCmap(lcc_levels,vik_cmap,[],False)
_, wcf_norm = CustomCmap(wcf_levels,vik_cmap,[],False)
_, wnd_norm = CustomCmap(wnd_levels,vik_cmap,[],False)

### Distribution plots

### Spatial plots
#### Set up

In [ ]:
if map_type:    
    graph_type_str = 'Map'# Linear or Map or Zonal
    plot_type = time_avg # 0: Monthly, 2: Seasonal, 3: All years
    num_yrs = 39

    ## Select plot type - yearly or monthly - to make and assign variables accordingly
    # Seasonal
    if plot_type == 2:
        avg_list = np.arange(4)
        date_str = np.array(['DJF','JJA','MAM','SON'])
        title_str = date_str 

    # All-data average
    elif plot_type == 3:
        avg_list = np.arange(1)
        date_str = np.array(['all_yrs'])
        title_str = np.array(['Ensemble mean for '+str(num_yrs)+' year average'])
        
    # Monthly
    elif plot_type == 0:
        avg_list = np.arange(12)
        date_str = np.array(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    
    # Yearly
    elif plot_type == 1:
        avg_list = np.arange(39)
        date_str = np.array([str(i+1979) for i in np.arange(1,40,1)])
        


#### Define graphing variables

In [ ]:
if map_type:     
    # Graphing variables if graphing type is the ensemble mean
    if ens_type == 'Mean':
        sbpt_shp = (1,2)
        figsz = (13,6)
        proj = ccrs.NorthPolarStereo()
        extent = [-180, 180, 50, 90]

    # Graphing variables if graphing type is 
    elif ens_type == 'All_members':
        sbpt_shp = (3,2)
        figsz = (11,12)
        proj = ccrs.NorthPolarStereo()
        extent = [-180, 180, 50, 90]

#### Calculate statistical signficance based on Wilks test

In [11]:
%%time
if map_type: 
    ## Calculate p-vals for t-test & Wilks field significance for all variables in varplot
    cri263K_data_avg = CalcStatSig(control_data_avg, control_data_std, n_control, cri263K_data_avg, cri263K_data_std, n_cri263K, var, sig, ens_type, time_avg)

CPU times: user 208 ms, sys: 28.7 ms, total: 237 ms
Wall time: 362 ms


In [12]:
%%time

# load computations into dataset
if map_type: 
    control_data_avg.compute()
    cri263K_data_avg.compute()

CPU times: user 7.19 ms, sys: 2.54 ms, total: 9.73 ms
Wall time: 9.75 ms


In [ ]:
if map_type:     
    # Plotting pandas Dataframe for spatial plots
    spdf = pd.DataFrame(data={'cmap':[cm.navia,plt.cm.jet,plt.cm.Blues,plt.cm.Blues,cm.lajolla_r,cm.lajolla_r,cm.lajolla_r,cm.lajolla_r,cm.roma_r,cm.roma_r,cm.navia,cm.navia],
                               'norm': [lcc_norm,tppr_norm,lwp_norm,iwp_norm,flux_norm,flux_norm,flux_norm,flux_norm,wcf_norm,wcf_norm,tppr_norm,tppr_norm],
                               'cnt_cbar_lb': ['LCC frequency','Surface temperature (K)','LWP (g/m$^{2}$)','IWP (g/m$^{2}$)','Downwelling longwave flux\nat the surface (W m$^{-2}$)',
                                               'F$_{\downarrow}$ (SW, surface) (W/m$^{2}$)',r'F$_{\uparrow}$ (LW, TOA) (W/m$^{2}$)',r'F$_{\uparrow}$ (SW, TOA) (W/m$^{2}$)',
                                               'LWCF (W/m$^{2}$)','SWCF (W/m$^{2}$)','Large-scale precipitation rate (mm/day)','Large-scale snow rate (mm/day)'],
                               'diff_cbar_lb': ['LCC frequency difference','Temperature difference (K)','LWP difference (g/m$^{2}$)','IWP difference (g/m$^{2}$)','Flux difference (W m$^{-2}$)',
                                                'Flux difference (W/m$^{2}$)','Flux difference (W/m$^{2}$)','Flux difference (W/m$^{2}$)','LWCF difference (W/m$^{2}$)','SWCF difference (W/m$^{2}$)',
                                                'Precipitation rate difference (mm/day)','Snow rate difference (mm/day)'],
                               'suptitle': ['Liquid-containing cloud frequency [LCC]','Surface temperature [TS]','Liquid water path [TGCLDLWP]','Ice water path [TGCLDIWP]',
                                            'Downwelling longwave flux at the surface [FLDS]','Downwelling shortwave flux at the surface [FSDS]','Upwelling longwave flux at the TOA [FLUT]',
                                            'Upwelling shortwave flux at the TOA [FSUTOA]','Longwave cloud forcing [LWCF]','Shortwave cloud forcing [SWCF]',
                                            'Large-scale precipitation rate [PRECL]','Large-scale snow rate [PRECSL]']},
                        index=['LCC','TS','TGCLDLWP','TGCLDIWP','FLDS','FSDS','FLUT','FSUTOA','LWCF','SWCF','PRECL','PRECSL'])


In [14]:
if ens_type == 'Mean' and map_type:
    ## Loop through all months/years
    for j in avg_list:
        if plot_type == 2:
            index = dict(season=j)
        elif plot_type >= 3:
            index = dict()

        print(date_str[j])
        cri263K_diff = cri263K_data_avg[var][index]-control_data_avg[var][index]

        print('max control-cri263K diff: '+str(cri263K_diff.loc[dict(lat=slice(50,90))].max().values))
        print('min control-cri263K diff: '+str(cri263K_diff.loc[dict(lat=slice(50,90))].min().values))

all_yrs
max control-cri263K diff: 2.3888855
min control-cri263K diff: -1.051178


In [15]:
if ens_type == 'Mean' and map_type:
    slice_i = dict(lat=slice(60,90))
    avg_dim = ('lat','lon')
    weights = np.cos(np.deg2rad(cri263K_diff.lat.loc[slice_i]))
    weights.compute()
    
    # Weight ensemble member data
    cri263K_diff_w = cri263K_diff.loc[slice_i].weighted(weights)
    
    # Calculate weighted mean
    cri263K_diff_avg_w = cri263K_diff_w.mean(avg_dim,skipna=True)

#### Make plots
##### Ensemble mean

In [16]:
%%time

if ens_type == 'Mean' and map_type:
    ## Loop through all months/years
    for j in avg_list:
        if plot_type == 2:
            index = dict(season=j)
        elif plot_type >= 3:
            index = dict()
        elif plot_type == 0:
            index = dict(month=j)
        elif plot_type == 1:
            index = dict(year=j)


        # Set up
        fig, axlist = plt.subplots(sbpt_shp[0],sbpt_shp[1],layout='constrained',subplot_kw=dict(projection=proj))
        fig.set_size_inches(6.9,3.4)

        if sig == 'Wilks':
            if plot_type >= 3:
                pcriti_cri263K = cri263K_data_avg['pcrit_'+var][index].values[0][0]
            else: 
                pcriti_cri263K = cri263K_data_avg['pcrit_'+var][index].values[0]

        # Plot data
        cax = control_data_avg[var][index].plot.contourf(
            ax=axlist[0],cmap=spdf.loc[var]['cmap'],levels=20,
            add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
        cax2 = (cri263K_data_avg[var][index]-control_data_avg[var][index]).plot.contourf(
            ax=axlist[1],add_colorbar=False,cmap=diff_cmap,
            norm=spdf.loc[var]['norm'],transform=ccrs.PlateCarree(),zorder=1)


        cri263K_data_avg['pvals_'+var][index].plot.contourf(
            ax=axlist[1],levels=[-0.01,pcriti_cri263K,1],hatches=['...',None],colors='none',
            add_colorbar=False,transform=ccrs.PlateCarree(),zorder=2)

        # Formatting
        axlist[0].coastlines(zorder=3)
        axlist[0].set_extent(extent, ccrs.PlateCarree())
        draw_circle(axlist[0], draw_major=False, draw_major_labels=False)
        axlist[0].set_title('(a) Control optics\n', fontsize=14)
        # gl0.xlabel_style = {'size':11}
        # gl0.ylabel_style = {'size':11}

        cb1 = fig.colorbar(cax,ax=axlist[0],pad=0.1,shrink=0.75,fraction=0.1)
        cb1.set_label(label=spdf.loc[var]['cnt_cbar_lb'], fontsize=12)
        cb1.ax.tick_params(labelsize=12)

        axlist[1].coastlines(zorder=3)
        axlist[1].set_extent(extent, ccrs.PlateCarree())
        draw_circle(axlist[1], draw_major=False, draw_major_labels=False)
        axlist[1].set_title('(b) 263 K-Control optics\n({:.2f}'.format(cri263K_diff_avg_w.values)+' W m$^{-2}$)', fontsize=14)
        # gl1.xlabel_style = {'size':11}
        # gl1.ylabel_style = {'size':11}

        cb2 = fig.colorbar(cax2,ax=axlist[1],pad=0.1,extend='both',shrink=0.75,fraction=0.1)
        cb2.set_label(label=spdf.loc[var]['diff_cbar_lb'], fontsize=12)
        cb2.ax.tick_params(labelsize=12)

        #fig.suptitle(spdf.loc[var]['suptitle']+' - '+title_str[j])

        fig.savefig(path_to_graphs+var+'.'+ens_type+'.'+graph_type_str+'.'+date_str[j]+'.pdf',dpi=300,bbox_inches='tight')

        plt.close(fig)
        axlist[0].clear()
        axlist[1].clear()

CPU times: user 4.98 s, sys: 28.9 ms, total: 5.01 s
Wall time: 5.51 s


##### All Ensemble members

In [17]:
%%time

if ens_type == 'All_members'and map_type:
    ## Loop through all months/years
    for j in avg_list:
        if plot_type == 2:
            index = dict(season=j)
        elif plot_type >= 3:
            index = dict()
        elif plot_type == 0:
            index = dict(month=j)
        elif plot_type == 1:
            index = dict(year=j)


        # Setup figure
        fig = plt.figure()
        fig.set_size_inches(figsz[0],figsz[1])

        axlist = []
        ens_num = 0
        for k in np.arange(1,7,2):
            # Set up axes
            ax1 = fig.add_subplot(sbpt_shp[0],sbpt_shp[1],k,projection=proj)
            ax2 = fig.add_subplot(sbpt_shp[0],sbpt_shp[1],k+1,projection=proj)
            axlist.append([ax1,ax2])

            # Modify index
            index['ensemble_member'] = ens_num

            if sig == 'Wilks':
                if np.logical_and(plot_type == 3,ens_type == 'All_members'):
                    pcriti_cri263K = cri263K_data_avg['pcrit_'+var][index].values[0]
                else:
                    pcriti_cri263K = cri263K_data_avg['pcrit_'+var][index].values

            # Plot data
            cax = control_data_avg[var][index].plot(
                    ax=ax1,cmap=spdf.loc[var]['cmap'],add_labels=False,
                    add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
            cax2 = (control_data_avg[var][index]-cri263K_data_avg[var][index]).plot.contourf(
                ax=ax2,add_colorbar=False,cmap=diff_cmap,add_labels=False,
                norm=spdf.loc[var]['norm'],transform=ccrs.PlateCarree(),zorder=1)

            cri263K_data_avg['pvals_'+var][index].plot.contourf(
                ax=ax2,levels=[-0.01,pcriti_cri263K,1],hatches=['\\\\',None],colors='none',
                add_colorbar=False,transform=ccrs.PlateCarree(),add_labels=False,zorder=2)

            # Formatting
            ax1.coastlines(zorder=3)
            ax1.set_extent(extent, ccrs.PlateCarree())
            gl1 = draw_circle(ax1, draw_major_labels=False)

            ax2.coastlines(zorder=3)
            ax2.set_extent(extent, ccrs.PlateCarree())
            gl2 = draw_circle(ax2, draw_major_labels=False)

            fig.colorbar(cax,ax=ax1,pad=0.1,label=spdf.loc[var]['cnt_cbar_lb'])

            fig.colorbar(cax2,ax=ax2,pad=0.1,label=spdf.loc[var]['diff_cbar_lb'],extend='both')

            ens_num += 1

        axlist = np.array(axlist)

        # Formatting figure labels
        axlist[0,0].set_title('Control',size=12)
        axlist[0,1].set_title('Control-263K optics',size=12)
        
        row_labels = ['Ensemble Member 1','Ensemble Member 2','Ensemble Member 3']

        for ax, row in zip(axlist[:,0],row_labels):
            ax.annotate(row,(0,0.5),xytext=(-45,0),ha='right',va='center',
                        size=12,rotation=90,xycoords='axes fraction',textcoords='offset points')

        #fig.suptitle(spdf.loc[var]['suptitle']+' - '+title_str[j])

        fig.savefig(path_to_graphs+var+'.'+ens_type+'.'+graph_type_str+'.'+date_str[j]+'.png',dpi=300)

        plt.close(fig)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 5.25 µs
